In [1]:
!pip install tensorflow-text

In [2]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download CellPhones -f CellPhonesRating.csv

 91% 110M/121M [00:00<00:00, 135MB/s] 
100% 121M/121M [00:00<00:00, 145MB/s]


In [3]:
from scipy.sparse import csr_matrix
import tensorflow as tf 
import numpy as np
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from tensorflow.keras import datasets, layers, models

In [4]:
df = pd.read_csv("/content/CellPhonesRating.csv.zip")
df = df.dropna(how='any',axis=0)
df = df.drop_duplicates(subset =['productID', 'reviewerID'] , keep = 'first')

In [5]:
df = df[:20000]

In [6]:
def DatasetToUserItemDataFrame(dataframe,userID,itemID,rating):
  #Setting new item IDs from string to int 
  itemKeys = [] 
  i = 0
  for item in dataframe.groupby(itemID).count()['Unnamed: 0'].values:
    temp = np.full((item),i)
    itemKeys = np.append(itemKeys,temp)
    i += 1

  #Setting new user IDs from string to int
  userKeysDic = {}
  userKeys = np.zeros((dataframe[userID].size))
  i = 0
  for user in dataframe[userID].unique():
    userKeysDic[user] = i
    i += 1
  i = 0
  for user in dataframe[userID]:
    userKeys[i] = userKeysDic[user]
    i += 1

  #Converting arrays from lfoat to int 
  userKeys = userKeys.astype(int)
  itemKeys = itemKeys.astype(int)

  
  user_item = csr_matrix((dataframe[rating].values.astype(int),(userKeys,itemKeys))) #Creating sparse matrix
  user_item_matrix = user_item.toarray() #Converting sparse matrix into array
  #df_user_item = pd.DataFrame(user_item_matrix,index = dataframe[userID].unique()  ,columns = dataframe[itemID].unique() ) 

  return user_item

In [7]:
user_item_matrix = DatasetToUserItemDataFrame(df,'reviewerID','productID','rating')

In [8]:
user_item_matrix.shape

(16029, 1364)

In [40]:
#Train/Test data split
from sklearn.model_selection import train_test_split
X_train, X_test= train_test_split(user_item_matrix)

In [41]:
X_train = X_train.toarray()

In [42]:
X_train.shape

(12021, 1364)

In [43]:
X_test.shape

(4008, 1364)

In [44]:
encoder_input = layers.Input(shape=(X_train.shape[1]),name='user_item')
flat = layers.Flatten()(encoder_input)
hid_encoder = layers.Dense(512,activation="relu")(flat)
encoder_output = layers.Dense(256,activation="relu")(hid_encoder)

decoder_input = layers.Dense(512,activation="relu")(encoder_output)
decoder_output = layers.Dense(X_train.shape[1],activation="relu")(decoder_input)

autoencoder = tf.keras.Model(inputs = encoder_input, outputs = decoder_output)

In [45]:
autoencoder.compile(optimizer = 'adam',
                    loss = 'mse',
                    metrics=['accuracy'])

In [ ]:
autoencoder.summary()

In [ ]:
autoencoder.fit(X_train, X_train, epochs = 5)